# Load to database

Rename and organize hazard, network and customer data.

Pre-requisites:
- python packages (`jupyter`, `psycopg2`, `fiona`).
- PostgreSQL database with PostGIS extension activated.
- database connection details saved to `config.json`

In [ ]:
import glob
import json

import psycopg2
import fiona
import geopandas

Load config from `config.json` (copy template from `config-template.json` and edit details).

In [ ]:
with open('config.json', 'r') as fh:
    config = json.load(fh)

Connect to the database.

In [ ]:
conn = psycopg2.connect(**config['database'])

## Create tables

Set up database tables for data loading:
- hazard outlines and scenarios
- infrastructure network nodes and edges
- boundaries and regions
- buildings

`hazard_modelled` table will contain modelled hazard extents

In [ ]:
with conn:
    with conn.cursor() as cur:
        cur.execute("DROP TABLE IF EXISTS hazard_modelled")
        cur.execute("""CREATE TABLE 
        hazard_modelled (
            id serial PRIMARY KEY,
            return_period int,
            model varchar(20),
            attributes jsonb,
            geom geometry(GEOMETRY,27700)
        );
        """)

`hazard_historic` table will contain historic hazard extents

In [ ]:
with conn:
    with conn.cursor() as cur:
        cur.execute("DROP TABLE IF EXISTS hazard_historic")
        cur.execute("""CREATE TABLE
        hazard_historic (
            id serial PRIMARY KEY,
            event varchar(20),
            start_date date,
            end_date date,
            geom geometry(GEOMETRY,27700)
        );
        """)

`hazard_scenarios` table will contain hazard scenarios derived from historic and modelled extens

In [ ]:
with conn:
    with conn.cursor() as cur:
        cur.execute("DROP TABLE IF EXISTS hazard_scenarios")
        cur.execute("""CREATE TABLE
        hazard_scenarios (
            id serial PRIMARY KEY,
            duration int,
            attributes jsonb,
            geom geometry(GEOMETRY,27700)
        );
        """)

`infrastructure_nodes` table will contain nodes from all infrastructure sectors

In [ ]:
with conn:
    with conn.cursor() as cur:
        cur.execute("DROP TABLE IF EXISTS infrastructure_nodes")
        cur.execute("""CREATE TABLE
        infrastructure_nodes (
            id serial PRIMARY KEY,
            original_id varchar(30),
            name varchar(40),
            infrastructure_sector varchar(10),
            centroid geometry(POINT,27700), -- single centroid
            footprint geometry(POLYGON,27700), -- service area/catchment
            geom geometry(GEOMETRY,27700), -- outline or point as provided
            attributes jsonb
        );
        """)

`infrastructure_edges` table will contain physical edges from infrastructure sectors where available (e.g. road, rail)

In [ ]:
with conn:
    with conn.cursor() as cur:
        cur.execute("DROP TABLE IF EXISTS infrastructure_edges")
        cur.execute("""CREATE TABLE
        infrastructure_edges (
            id serial PRIMARY KEY,
            original_id varchar(30),
            name varchar(40),
            from_node_id int REFERENCES infrastructure_nodes (id),
            to_node_id int REFERENCES infrastructure_nodes (id),
            infrastructure_sector varchar(10),
            geom geometry(LINESTRING,27700), -- line as provided
            attributes jsonb
        );
        """)

`interdependency_edges` table will contain non-physical or inferred edges of the infrastructure network (e.g. electricity distribution)

In [ ]:
with conn:
    with conn.cursor() as cur:
        cur.execute("DROP TABLE IF EXISTS interdependency_edges")
        cur.execute("""CREATE TABLE
        interdependency_edges (
            id serial PRIMARY KEY,
            original_id varchar(30),
            from_node_id int REFERENCES infrastructure_nodes (id),
            to_node_id int REFERENCES infrastructure_nodes (id),
            infrastructure_sector varchar(10),
            geom geometry(LINESTRING,27700), -- straight line from_node->to_node
            attributes jsonb
        );
        """)

`zones` table will contain administrative zones if/when available (e.g. national boundaries, Flood Risk Management Systems)

In [ ]:
with conn:
    with conn.cursor() as cur:
        cur.execute("DROP TABLE IF EXISTS zones")
        cur.execute("""CREATE TABLE
        zones (
            id serial PRIMARY KEY,
            original_id varchar(30),
            name varchar(40),
            zone_type varchar(20),
            geom geometry(GEOMETRY,27700)
        );
        """)

`buildings` table will contain residential and commercial buildings

In [ ]:
with conn:
    with conn.cursor() as cur:
        cur.execute("DROP TABLE IF EXISTS buildings")
        cur.execute("""CREATE TABLE
        buildings (
            id serial PRIMARY KEY,
            uprn int,
            floor_area real,
            geom geometry(POINT,27700),
            attributes jsonb
        );
        """)

## Load data

Load data as provided, transforming/renaming as necessary
- modelled floods
- historic floods
- airports
- electricity cables, overhead lines, substations, towers
- gas sites, pipes
- ports
- railway lines, stations
- road links, nodes
- wastewater treatment sites
- water treatment works
- residential and non-residential buildings

In [ ]:
filenames = glob.glob('../Data/Flooding data/LTISII_RFO/RecordedFloodOutlines_Final/*.shp')

In [ ]:
filenames

In [ ]:
df = geopandas.read_file(filenames[0])[
    [        
        'OUTLINE_CO',
        'NAME',
        'EVENT_CODE',
        'START_DATE',
        'END_DATE',
        'FLOOD_SRC',
        'FLOOD_CAUS',
        'FLUVIAL_IN',
        'TIDAL_IND',
        'COASTAL_IN',
        'HFM_IND',
        'geometry'
    ]
]

df.rename(columns={
    'EVENT_CODE': 'event_code',
    'OUTLINE_CO': 'id',
    'NAME': 'name',
    'START_DATE': 'start_date',
    'END_DATE': 'end_date',
    'FLOOD_SRC': 'flood_source',
    'FLOOD_CAUS': 'flood_cause',
    'FLUVIAL_IN': 'fluvial',
    'TIDAL_IND': 'tidal',
    'COASTAL_IN': 'coastal',
    'HFM_IND': 'hfm',
    'geometry': 'geom'
})

In [ ]:
df